In [3]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import mnist_model

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')
mnist_path = './mnist.pth'

def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset = MNIST('./data', transform=img_transform,download = False)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)
mnist = mnist_model.mnist(pretrained=True)

In [4]:
for epoch in range(num_epochs):
    eval_acc = 0.
    mnist.eval()
    for data in dataloader:
        img, label = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # ===================mnist test==================
        out = mnist(output)
        _, pred = torch.max(out, 1)
        eval_acc += (pred == label).float().mean()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}, Acc:{:.6f}'
          .format(epoch + 1, num_epochs, loss.item(),eval_acc/len(dataloader)))
#     print( f'Acc: {eval_acc/len(dataloader):.6f}')
    
    if epoch % 5 == 0:
        
        pic = to_img(output[0:8].cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/100], loss:0.5276, Acc:0.246979
epoch [2/100], loss:0.5101, Acc:0.473747


KeyboardInterrupt: 